<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/AggregateDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('AggregateDF').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 16.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=11d86dca6075c0a5b811b26aeac68aa5e61b18cc2e6f0b141a8df66d5fd783a0
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
You are working with 1 core(s)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/PySpark/Datasets/'
airbnb = spark.read.csv(path + 'nyc_air_bnb.csv', inferSchema=True, header=True)

Mounted at /content/drive


In [5]:
airbnb.show(5,False)

+----+------------------------------------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|id  |name                                            |host_id|host_name  |neighbourhood_group|neighbourhood|latitude|longitude|room_type      |price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+------------------------------------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt home by the park              |2787   |John       |Brooklyn           |Kensington   |40.64749|-73.97237|Private room   |149  |1             |9                |2018-10-19 |0.21             |6

In [4]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))

print(df.printSchema())
df.show(5,False)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None
+----+------------------------------------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|id  |name         

In [11]:
# For example we may be interested to see how many listings there were per neighbourhood group. 
df.groupBy("neighbourhood_group").count().orderBy(desc('count')).show(100)

+--------------------+-----+
| neighbourhood_group|count|
+--------------------+-----+
|           Manhattan|21594|
|            Brooklyn|20055|
|              Queens| 5630|
|               Bronx| 1080|
|       Staten Island|  370|
|                null|  185|
|              Harlem|   13|
|  Bedford-Stuyvesant|    9|
|      Hell's Kitchen|    7|
|     Upper East Side|    7|
|            Elmhurst|    7|
|        Williamsburg|    6|
|         East Harlem|    5|
|          Mott Haven|    5|
|        East Village|    4|
|  Washington Heights|    4|
|             Midtown|    4|
|            Canarsie|    4|
|            Bushwick|    4|
|     Upper West Side|    3|
|           Woodhaven|    3|
|             Midwood|    3|
|    Ditmars Steinway|    3|
|            Flushing|    3|
|       Crown Heights|    3|
|    Long Island City|    3|
| Morningside Heights|    3|
|            Longwood|    2|
|             Astoria|    2|
|       East Elmhurst|    2|
|Prospect-Lefferts...|    2|
|      Pelham 

In [12]:
# Groupby Function with count (you can also use sum, min, max)
df.groupBy("neighbourhood_group").min('price').show(5)

+-------------------+----------+
|neighbourhood_group|min(price)|
+-------------------+----------+
|         Douglaston|         1|
|             Queens|        10|
|              Nadia|      null|
|            Midtown|         2|
|    Jackson Heights|         2|
+-------------------+----------+
only showing top 5 rows



In [13]:
df.groupBy("neighbourhood").agg({'price': 'mean'}).show(5)

+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



In [14]:
from pyspark.sql.functions import *
df.groupBy("neighbourhood").agg( min(df.price).alias("Min Price"), max(df.price).alias("Max Price")).show(5)

+-------------+---------+---------+
|neighbourhood|Min Price|Max Price|
+-------------+---------+---------+
|       Corona|       23|      359|
| Richmondtown|       78|       78|
| Prince's Bay|       85|     1250|
|  Westerleigh|       40|      103|
|   Mill Basin|       85|      299|
+-------------+---------+---------+
only showing top 5 rows



In [28]:
summary = df.select('price', 'minimum_nights', 'number_of_reviews').summary('count', 'min', 'max', 'mean')
summary.show(5)

+-------+------------------+------------------+-----------------+
|summary|             price|    minimum_nights|number_of_reviews|
+-------+------------------+------------------+-----------------+
|  count|             48887|             48891|            48738|
|    min|               -74|                 0|                0|
|    max|             10000|              1250|              629|
|   mean|152.22298361527604|7.1286126280910596|23.25827896097501|
+-------+------------------+------------------+-----------------+



In [32]:
from pyspark.sql.functions import *
df.agg( min(df.price).alias("Min Price"), max(df.price).alias("Max Price")).show()
df.select( countDistinct("neighbourhood_group").alias('CountD'), avg('price').alias('avg_P'), stddev("price").alias('stdev_P')).show()
df.agg( {'number_of_reviews': 'max'} ).withColumnRenamed("max(number_of_reviews)", "Max Reviews").show()

+---------+---------+
|Min Price|Max Price|
+---------+---------+
|      -74|    10000|
+---------+---------+

+------+------------------+-----------------+
|CountD|             avg_P|          stdev_P|
+------+------------------+-----------------+
|    77|152.22298361527604|238.5414668883948|
+------+------------------+-----------------+

+-----------+
|Max Reviews|
+-----------+
|        629|
+-----------+



In [ ]:
# Pivot Function
# https://www.jianshu.com/p/36bdf156cbda